In [ ]:
# import capytaine as cpy
import autograd.numpy as np
import matplotlib.pyplot as plt
import xarray as xr
# from math import comb

import wecopttool as wot
from wecopttool.waves import jonswap_spectrum as js
from wecopttool.waves import long_crested_wave as lcw
from wecopttool.waves import omnidirectional_spectrum

import os
import Pioneer_Inverted_Pendulum as pip
from scipy.optimize import Bounds

## set colorblind-friendly colormap for plots
plt.style.use('tableau-colorblind10')

wot.set_loglevel('ERROR', capytaine=False)
data_path = os.path.join('pioneer_data')

In [ ]:
wavefreq = 0.375
f1 = wavefreq
nfreq = 5
amplitude = 0.2

waves = wot.waves.regular_wave(f1, nfreq, wavefreq, amplitude)

In [ ]:


cntrl_type = 'unstructured'
NPIP = pip.NonlinearInvertedPendulumPTO(f1, nfreq, ndof = 1, control_type = cntrl_type)



LinPIP = pip.LinearizedInvertedPendulumPTO(f1, nfreq, ndof = 1, control_type = cntrl_type)
wec_lin = pip.PioneerBuoy.from_empirical_data(f1, nfreq, f_add = LinPIP.f_add, constraints = LinPIP.constraints)  
wec_nl = pip.PioneerBuoy.from_empirical_data(f1, nfreq, f_add = NPIP.f_add, constraints = NPIP.constraints)
res_lin = LinPIP.solve(wec_lin, waves)
x_wec_0, x_opt_0 = wec_lin.decompose_state(res_lin[0].x)
res_nl = NPIP.solve(wec_nl, waves, x_wec_0, x_opt_0)


In [ ]:
nsubsteps = 4
wec_fdom_list, wec_tdom_list, pen_fdom_list, pen_tdom_list = NPIP.post_process(wec_nl, res_nl, waves, nsubsteps=nsubsteps)
wec_fdom, wec_tdom, pen_fdom, pen_tdom = wec_fdom_list[0], wec_tdom_list[0], pen_fdom_list[0], pen_tdom_list[0]

In [ ]:
ind_freqs = np.arange(nfreq+1)

In [ ]:
ax = plt.figure().add_subplot(projection='3d')
x, y, z = pen_tdom['rel_vel'], pen_tdom['rel_pos'], pen_tdom['torque'].sel(type = 'Generator')

ax.plot(x,y, z, color = 'C2', linestyle = '--')
ax.set_xlabel(x.attrs['long_name']+'['+x.attrs['units']+']')
ax.set_ylabel(y.attrs['long_name']+'['+y.attrs['units']+']')
ax.set_zlabel(z.attrs['long_name']+'['+z.attrs['units']+']')

plt.tight_layout()

## 2D amplitude and wavefreq

In [ ]:
wave_freq_vec = [0.2, 0.25, 0.275, 0.3, 0.325, 0.35, 0.375, 0.4, 0.45, 0.5]
# wave_freq_vec = [ 0.325,  0.375,  0.425]
# wave_freq_vec = [0.375,  0.425]

nfreq = 5
# amplitude_vec = [0.05]
amplitude_vec = [0.1]

# cntrl_type_vec = ['PI', 'unstructured']
cntrl_type_vec = ['P', 'PI', 'unstructured']

#create empty dataset for results
results = []

for cntrl_type in cntrl_type_vec:
    for amplitude in amplitude_vec:
        for wavefreq in wave_freq_vec:
            f1_reg = wavefreq
            waves_reg = wot.waves.regular_wave(f1_reg, nfreq, wavefreq, amplitude)
            NPIP = pip.NonlinearInvertedPendulumPTO(f1_reg, nfreq, ndof = 1, control_type = cntrl_type)
            LinPIP = pip.LinearizedInvertedPendulumPTO(f1_reg, nfreq, ndof = 1, control_type = cntrl_type)
            wec_lin = pip.PioneerBuoy.from_empirical_data(f1_reg, nfreq, f_add = LinPIP.f_add, constraints = LinPIP.constraints)  
            wec_nl = pip.PioneerBuoy.from_empirical_data(f1_reg, nfreq, f_add = NPIP.f_add, constraints = NPIP.constraints)
            try:
                res_lin = LinPIP.solve(wec_lin, waves_reg)
                x_wec_0, x_opt_0 = wec_lin.decompose_state(res_lin[0].x)
                res_nl = NPIP.solve(wec_nl, waves_reg, x_wec_0, x_opt_0)
                nsubsteps = 5
                wec_fdom_list, wec_tdom_list, pen_fdom_list, pen_tdom_list = NPIP.post_process(wec_nl, res_nl, waves_reg, nsubsteps=nsubsteps)
                wec_fdom, wec_tdom, pen_fdom, pen_tdom = wec_fdom_list[0], wec_tdom_list[0], pen_fdom_list[0], pen_tdom_list[0]
                epower_fd = np.real(-(1/2)*np.conj(pen_fdom['quad_current'])*pen_fdom['back_emf'])
                Pe_avg = np.real(-1*pen_fdom['epower'])
                results.append({
                    'Pavg': Pe_avg[0],
                    'harm_dis':1-epower_fd[1].data/Pe_avg[0].data,
                    'rel_vel': pen_tdom['rel_vel'],
                    'rel_pos': pen_tdom['rel_pos'],
                    'torque': pen_tdom['torque'].sel(type = 'Generator'),
                    'cntrl_type': cntrl_type,
                    'amplitude': amplitude,
                    'wavefreq': wavefreq
                })
            except:
                print(f"An exception occurred for {cntrl_type} control, A={amplitude}, f={wavefreq}, adding nan") 
                results.append({
                    'Pavg': np.nan,
                    'harm_dis': np.nan,
                    'rel_vel': np.nan,
                    'rel_pos': np.nan,
                    'torque': np.nan,
                    'cntrl_type': cntrl_type,
                    'amplitude': amplitude,
                    'wavefreq': wavefreq
                })



In [ ]:
import pickle
from datetime import datetime

subdir = 'pioneer_res'
os.makedirs(subdir, exist_ok=True)  # Create the directory if it doesn't exist
formatted_time = datetime.now().strftime("%Y-%m-%d_%H-%M")
filename = os.path.join(subdir, f'result_list_A0p10m_{formatted_time}')
with open(filename, 'wb') as f:
    pickle.dump(results, f)  # Save as a pickle file


In [ ]:
with open('pioneer_res\\result_list_A0p05m_2025-02-26_13-37', 'rb') as f:
    resA0p05 = pickle.load(f)
resA0p05

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.cm import ScalarMappable


my_blues = plt.cm.Blues(np.linspace(0.4, 0.75, len(wave_freq_vec)))
my_reds = plt.cm.Reds(np.linspace(0.4, 0.75, len(wave_freq_vec)))
color_mix = []
if len(amplitude_vec)> 1:
    for n in np.arange(len(amplitude_vec)):
        ratio = n/(len(amplitude_vec)-1)
        color_mix.append(my_blues*ratio+my_reds*(1-ratio))
else:
    color_mix.append(my_blues)

fig = plt.figure(figsize=[len(cntrl_type_vec)*4, 8])
ax_list = []
for kk, cntrl_type in enumerate(cntrl_type_vec):
    ax_list.append(fig.add_subplot(1, len(cntrl_type_vec), kk+1, projection='3d'))
    ax_list[kk].set_title(f'Control: {cntrl_type}')
    for i, A in enumerate(amplitude_vec):
        for j, wf in enumerate(wave_freq_vec):
            idx = kk * len(cntrl_type_vec) + i * len(amplitude_vec) + j  # Calculate the correct index
            res_ind = results[idx]
            x, y, z = res_ind['rel_vel'], res_ind['rel_pos'], res_ind['torque']
            if res_ind['harm_dis'] >0:
                ax_list[kk].plot(x,y, z, color = color_mix[i][j])
        # for i, A in enumerate(amplitude_vec):
        sm = ScalarMappable(cmap=LinearSegmentedColormap.from_list(name = 'Custom', colors=color_mix[i], N=len(color_mix[i])))#, norm=norm)   
        cbar = fig.colorbar(sm, ax = ax_list[kk], orientation='horizontal', shrink=0.08, aspect = 3, ticks = [], label = f'A={A}m', anchor = (1.2, 4+0.1*i))
    ax_list[kk].set_xlabel(x.attrs['long_name']+'['+x.attrs['units']+']')
    ax_list[kk].set_ylabel(y.attrs['long_name']+'['+y.attrs['units']+']')
    ax_list[kk].set_zlabel(z.attrs['long_name']+'['+z.attrs['units']+']')


    # ax.set_xlim([-1, 1])
    # ax.set_ylim([-0.5, 0.5])
    # ax.set_zlim([-5, 5])



# Step 2: Determine global limits using get_xlim, get_ylim, get_zlim
x_limits = [np.inf, -np.inf]
y_limits = [np.inf, -np.inf]
z_limits = [np.inf, -np.inf]

for ax in ax_list:
    x_limits[0] = min(x_limits[0], ax.get_xlim()[0])
    x_limits[1] = max(x_limits[1], ax.get_xlim()[1])
    y_limits[0] = min(y_limits[0], ax.get_ylim()[0])
    y_limits[1] = max(y_limits[1], ax.get_ylim()[1])
    z_limits[0] = min(z_limits[0], ax.get_zlim()[0])
    z_limits[1] = max(z_limits[1], ax.get_zlim()[1])

# Step 3: Set the same limits for all axes
for ax in ax_list:
    ax.set_xlim(x_limits)
    ax.set_ylim(y_limits)
    ax.set_zlim(z_limits)






In [ ]:



fig, axes = plt.subplots(2, len(cntrl_type_vec), tight_layout=True, figsize=[len(cntrl_type_vec)*4, 4], sharex=True, sharey= 'row')

Pavg_mat = np.zeros((len(cntrl_type_vec), len(amplitude_vec), len(wave_freq_vec)))
Rspec_mat = np.zeros((len(cntrl_type_vec), len(amplitude_vec), len(wave_freq_vec)))
for kk, cntrl_type in enumerate(cntrl_type_vec):
    axes[0,kk].set_title(f'Control: {cntrl_type}')



    # Loop through amplitudes and wave frequencies
    for i, A in enumerate(amplitude_vec):
        for j, wf in enumerate(wave_freq_vec):
            idx = kk * len(cntrl_type_vec) + i * len(amplitude_vec) + j  # Calculate the correct index
            res_ind = results[idx]
            P_avg = res_ind['Pavg']
            Rspec = res_ind['harm_dis']

            if Rspec >0:
                Pavg_mat[kk,i,j] = P_avg
                Rspec_mat[kk,i,j] = Rspec
                axes[0,kk].plot(wf, Rspec, '.', color = color_mix[i][j])
                axes[1,kk].plot(wf, P_avg, 'o', color = color_mix[i][j])
            else:
                Pavg_mat[kk,i,j] = np.nan
                Rspec_mat[kk,i,j] = np.nan
        sm = ScalarMappable(cmap=LinearSegmentedColormap.from_list(name = 'Custom', colors=color_mix[i], N=len(color_mix[i])))#, norm=norm)   
        cbar = fig.colorbar(sm, ax = axes[0,kk], orientation='horizontal', shrink=0.08, aspect = 3, ticks = [], label = f'A={A}m', anchor = (1.2, 0.1+0.1*i))

for kk, cntrl_type in enumerate(cntrl_type_vec):
    for i, A in enumerate(amplitude_vec):
        axes[0,kk].plot(wave_freq_vec,Rspec_mat[kk,i,:], linestyle = '--', color = [0,0,0,0.5])
        axes[1,kk].plot(wave_freq_vec,Pavg_mat[kk,i,:], linestyle = '--', color = [0,0,0,0.5])

[ax.set_xlabel('Reg. wave freq (Hz)') for ax in axes[1,:].flatten()]
axes[1,0].set_ylabel('Avg. power (W)')
axes[0,0].set_ylabel('Harmonic distortion')
[ax.grid() for ax in axes.flatten()]





In [ ]:
# Create a figure and subplots
fig = plt.figure(figsize=(10, 10))

# First subplot
ax1 = fig.add_subplot(2, 2, 1, projection='3d')


# X, Y = np.meshgrid(amplitude_vec, wave_freq_vec)
X, Y = np.meshgrid(wave_freq_vec, amplitude_vec)

# ax1.scatter(X, Y, P_mat, c= P_mat, cmap='viridis')
ax1.plot_surface(X, Y, Rspec_mat, cmap='viridis')

ax2 = fig.add_subplot(2, 2, 2, projection='3d')
ax2.plot_surface(X, Y, Pavg_mat, cmap='viridis')

ax1.set_xlabel('Reg. wave freq (Hz)')
ax2.set_xlabel('Reg. wave freq (Hz)')

ax1.set_ylabel('Reg. wave amplitude (m)')
ax2.set_ylabel('Reg. wave amplitude (m)')

ax1.set_zlabel('Harmonic distortion')
ax2.set_zlabel('Avg. power (W)')
